In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import date
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

In [3]:
train.head(10)

Deal_title                         Lead_name                  Industry  \
0  TitleM5DZY     Davis, Perkins and Bishop Inc               Restaurants   
1  TitleKIW18                    Bender PLC LLC     Construction Services   
2  TitleFXSDN             Carter-Henry and Sons         Hospitals/Clinics   
3  TitlePSK4Y                    Garcia Ltd Ltd               Real Estate   
4  Title904GV                  Lee and Sons PLC        Financial Services   
5  Title00VOR                    Chavez Ltd Inc                     Banks   
6  TitleOZQRY           Williamson LLC and Sons                     Banks   
7  TitleV05WV  Livingston, York and Adams Group  Architecture/Engineering   
8  TitleC2RNN                    Powers Ltd Inc        Education/Training   
9  TitleDZT40                    Shea Group PLC                      REIT   

  Deal_value Weighted_amount Date_of_creation      Pitch  \
0    320506$      2067263.7$       2020-03-29  Product_2   
1     39488$       240876.8$       2019-07-10  Product_2   
2    359392$      2407926.4$       2019-07-27  Product_1   
3     76774$       468321.4$       2021-01-30  Product_2   
4    483896$             NaN       2019-05-22  Product_2   
5    418674$      2637646.2$       2019-06-30  Product_1   
6    384356$      2709709.8$       2019-11-20  Product_2   
7    245205$      1642873.5$       2020-12-22  Product_2   
8    343280$      1991024.0$       2019-01-08  Product_1   
9    293611$      1791027.1$       2020-07-04  Product_1   

             Contact_no             Lead_revenue Fund_category  ...  \
0          607.447.7883         50 - 100 Million    Category 2  ...   
1          892-938-9493  500 Million - 1 Billion    Category 4  ...   
2          538.748.2271  500 Million - 1 Billion    Category 4  ...   
3   (692)052-1389x75188  500 Million - 1 Billion    Category 3  ...   
4  001-878-814-6134x015         50 - 100 Million    Category 3  ...   
5     (418)259-9934x952         50 - 100 Million    Category 2  ...   
6         (881)077-4692  500 Million - 1 Billion    Category 1  ...   
7          953.762.9149  500 Million - 1 Billion    Category 1  ...   
8         (566)194-6345        100 - 500 Million    Category 3  ...   
9         (292)819-5746  500 Million - 1 Billion    Category 3  ...   

                                         Designation  \
0                           Executive Vice President   
1                             Chairman/CEO/President   
2                               SVP/General Counsel    
3                            CEO/Co-Founder/Chairman   
4                           Executive Vice President   
5                            CEO/Co-Founder/Chairman   
6                            CEO/Co-Founder/Chairman   
7  Vice President / GM (04-present) : VP Sales an...   
8                  Chairman/Chief Innovation Officer   
9                                                CEO   

                    Lead_POC_email                  Hiring_candidate_role  \
0         charlenewerner@davis.com                   Community pharmacist   
1            terrylogan@bender.com                 Recruitment consultant   
2    arielhamilton@carterhenry.com                 Health service manager   
3            erinwilson@garcia.com         Therapist, speech and language   
4           mr.christopher@lee.com                          Media planner   
5         crystalchavez@chavez.com                         Microbiologist   
6         saradixon@williamson.com                           Cartographer   
7  christianstanley@livingston.com             Engineer, maintenance (IT)   
8        josephthompson@powers.com  Product/process development scientist   
9           williamgrimes@shea.com                  Engineering geologist   

       Lead_source Level_of_meeting                 Last_lead_update  \
0          Website          Level 3                         No track   
1           Others          Level 1  Did not hear back after Level 1   
2  Marketing Event          Lev

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Deal_title             7007 non-null   object 
 1   Lead_name              7007 non-null   object 
 2   Industry               7006 non-null   object 
 3   Deal_value             6956 non-null   object 
 4   Weighted_amount        6482 non-null   object 
 5   Date_of_creation       7007 non-null   object 
 6   Pitch                  7007 non-null   object 
 7   Contact_no             7007 non-null   object 
 8   Lead_revenue           7007 non-null   object 
 9   Fund_category          7007 non-null   object 
 10  Geography              6035 non-null   object 
 11  Location               6996 non-null   object 
 12  POC_name               6999 non-null   object 
 13  Designation            7007 non-null   object 
 14  Lead_POC_email         7007 non-null   object 
 15  Hiri

# Clean train data

In [5]:
train = train.drop(['Deal_title', 'Lead_name', 'Contact_no', 'Lead_POC_email'], axis = 1)

In [6]:
train['Date_of_creation'] = pd.to_datetime(train['Date_of_creation'])
train['days_since_created'] = train['Date_of_creation'].apply(lambda x:(pd.Timestamp('today') - x).days)
train.drop(['Date_of_creation'], inplace=True, axis=1)

In [7]:
train.Deal_value = train.Deal_value.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)
train.Weighted_amount = train.Weighted_amount.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)

In [8]:
train['min_revenue'] = train.Lead_revenue.apply(lambda x: x.split('-')[0])
train['max_revenue'] = train.Lead_revenue.apply(lambda x: x.split('-')[1])
train = train.drop(['Lead_revenue'], axis = 1)

In [9]:
train.min_revenue = train.min_revenue.apply(lambda x: '500' if x == '500 Million ' else x)
train.max_revenue = train.max_revenue.apply(lambda x: '100' if x == ' 100 Million' else ('500' if x == ' 500 Million' else '1000'))

In [10]:
train["Deal_value"] = train.Deal_value.astype(float)
train["Weighted_amount"] = train.Weighted_amount.astype(float)
train["min_revenue"] = train.min_revenue.astype(float)
train["max_revenue"] = train.max_revenue.astype(float)

In [11]:
train['Weighted_amount'] = train['Weighted_amount'].fillna((train['Weighted_amount'].mean()))
train['Deal_value'] = train['Deal_value'].fillna((train['Deal_value'].mean()))

In [12]:
train['Geography'] = train['Geography'].fillna(method = 'ffill')
train['Resource'] = train['Resource'].fillna(method = 'bfill')

In [13]:
def fill(train, s):
    n = 7007
    x = 'No'
    for i in range(n):
        if train[s][i] == '?':
            train[s][i] = x
        x = train[s][i]
    return train
train = fill(train, 'Last_lead_update')

<ipython-input-13-08feb23ec113>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[s][i] = x


In [14]:
train.Success_probability = train.Success_probability.apply(lambda x: -x if x < 0 else x)

In [15]:
train.describe()

Deal_value  Weighted_amount  Internal_rating  Success_probability  \
count    7007.000000     7.007000e+03      7007.000000          7007.000000   
mean   249656.025446     1.569884e+06         3.009562            64.845034   
std    144231.008181     8.863772e+05         1.418666            17.566890   
min      1551.000000     8.708000e+03         1.000000             5.000000   
25%    123293.500000     8.264284e+05         2.000000            60.600000   
50%    248796.000000     1.569884e+06         3.000000            65.300000   
75%    375706.500000     2.293031e+06         4.000000            69.600000   
max    500000.000000     3.601416e+06         5.000000           107.340000   

       days_since_created  min_revenue  max_revenue  
count         7007.000000  7007.000000  7007.000000  
mean           448.625089   219.430569   539.517625  
std            224.690222   202.070367   367.838423  
min             60.000000    50.000000   100.000000  
25%            255.500000    50.000000   100.000000  
50%            448.000000   100.000000   500.000000  
75%            643.000000   500.000000  1000.000000  
max            836.000000   500.000000  1000.000000

In [16]:
X = train.drop(['Success_probability'], axis=1)
y = train.Success_probability

In [17]:
enc = OneHotEncoder(handle_unknown='ignore')
X = pd.DataFrame(enc.fit_transform(X).toarray())

In [18]:
X.head()

0      1      2      3      4      5      6      7      8      9      ...  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   

   20940  20941  20942  20943  20944  20945  20946  20947  20948  20949  
0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0  
2    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0  
3    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0  
4    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0    0.0  

[5 rows x 20950 columns]

# clean test data

In [19]:
Deal_title = test.Deal_title
test = test.drop(['Deal_title', 'Lead_name', 'Contact_no', 'Lead_POC_email'], axis = 1)

In [20]:
test['Date_of_creation'] = pd.to_datetime(test['Date_of_creation'])
test['days_since_created'] = test['Date_of_creation'].apply(lambda x:(pd.Timestamp('today') - x).days)
test.drop(['Date_of_creation'], inplace=True, axis=1)

In [21]:
test.Deal_value = test.Deal_value.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)
test.Weighted_amount = test.Weighted_amount.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)

In [22]:
test['min_revenue'] = test.Lead_revenue.apply(lambda x: x.split('-')[0])
test['max_revenue'] = test.Lead_revenue.apply(lambda x: x.split('-')[1])
test = test.drop(['Lead_revenue'], axis = 1)

In [23]:
test.min_revenue = test.min_revenue.apply(lambda x: '500' if x == '500 Million ' else x)
test.max_revenue = test.max_revenue.apply(lambda x: '100' if x == ' 100 Million' else ('500' if x == ' 500 Million' else '1000'))

In [24]:
test["Deal_value"] = test.Deal_value.astype(float)
test["Weighted_amount"] = test.Weighted_amount.astype(float)
test["min_revenue"] = test.min_revenue.astype(float)
test["max_revenue"] = test.max_revenue.astype(float)

In [25]:
test['Weighted_amount'] = test['Weighted_amount'].fillna((test['Weighted_amount'].mean()))
test['Deal_value'] = test['Deal_value'].fillna((test['Deal_value'].mean()))

In [26]:
test['Geography'] = test['Geography'].fillna(method = 'ffill')
test['Resource'] = test['Resource'].fillna(method = 'bfill')

In [27]:
def fill(test, s):
    n = 2093
    x = 'No'
    for i in range(n):
        if test[s][i] == '?':
            test[s][i] = x
        x = test[s][i]
    return test
test = fill(test, 'Last_lead_update')

<ipython-input-27-d90b6ffb99be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[s][i] = x


In [28]:
test.Internal_rating = test.Internal_rating.apply(lambda x: -x if x < 0 else x)

In [29]:
test.describe()

Deal_value  Weighted_amount  Internal_rating  days_since_created  \
count    2093.000000     2.093000e+03      2093.000000         2093.000000   
mean   248052.138889     1.556917e+06         3.189221          449.614907   
std    141862.423966     8.873994e+05         4.478122          223.069168   
min      2025.000000     1.316250e+04         1.000000           60.000000   
25%    126390.000000     8.049900e+05         2.000000          259.000000   
50%    250650.000000     1.556917e+06         3.000000          449.000000   
75%    372023.000000     2.311754e+06         4.000000          644.000000   
max    499392.000000     3.409163e+06        82.340000          836.000000   

       min_revenue  max_revenue  
count  2093.000000  2093.000000  
mean    213.186813   525.704730  
std     200.502528   368.447914  
min      50.000000   100.000000  
25%      50.000000   100.000000  
50%     100.000000   500.000000  
75%     500.000000  1000.000000  
max     500.000000  1000.000000

In [30]:
test = pd.DataFrame(enc.transform(test).toarray())

In [31]:
test.head()

0      1      2      3      4      5      6      7      8      9      ...  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
1    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
2    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   
4    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   

   20940  20941  20942  20943  20944  20945  20946  20947  20948  20949  
0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0  
1    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0  
2    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0  
3    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0    0.0  
4    0.0    0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    1.0  

[5 rows x 20950 columns]

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
model = RandomForestRegressor(criterion="mae", max_features="sqrt", n_jobs=-1, n_estimators=100)
model.fit(X_train, y_train)
pred = model.predict(X_val)
score = max(0, 100-np.sqrt(mean_squared_error(y_val, pred)))
score

82.79187444798094

In [34]:
model1 = RandomForestRegressor(criterion="mse", max_features="sqrt", n_jobs=-1, n_estimators=100)
model1.fit(X_train, y_train)
pred1 = model1.predict(X_val)
score1 = max(0, 100-np.sqrt(mean_squared_error(y_val, pred1)))
score1

82.76838805792767

In [35]:
model2 = RandomForestRegressor(criterion="mae", max_features="log2", n_jobs=-1, n_estimators=100)
model2.fit(X_train, y_train)
pred2 = model2.predict(X_val)
score2 = max(0, 100-np.sqrt(mean_squared_error(y_val, pred2)))
score2

82.70172184023973

In [39]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)
pred3 = regressor.predict(X_val)
score3 = max(0, 100-np.sqrt(mean_squared_error(y_val, pred3)))
score3

79.39670048445524

In [36]:
test_pred = model.predict(test)

In [37]:
output = pd.DataFrame({'Deal_title': Deal_title,
                       'Success_probability': test_pred})

In [38]:
output.to_csv('submission.csv', index=False)